In [39]:
!nvidia-smi

Wed Jul 26 08:59:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   31C    P0    43W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Setup environment

In [40]:
workspace_dir = '/nfs/Workspace/TeethSeg'
%cd {workspace_dir}
%load_ext autoreload
%autoreload 2

/nfs/Workspace/TeethSeg
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preprocess

In [42]:
# !PYTHONPATH={workspace_dir} /opt/conda/envs/TeethSeg/bin/python {workspace_dir}/expers/preprocess.py

## Setup config

In [43]:
!pwd

/nfs/Workspace/TeethSeg


In [49]:
import os

model_name = 'cswinunetr_96_3_mask'
data_name = 'tooth'
exp_name = 'exp_test'


root_exp_dir = os.path.join(
    workspace_dir, 
    'exps',
    model_name,
    data_name,
    exp_name
)

data_dir = '/nfs/Workspace/dataset/teeth/data50'
data_json = os.path.join(data_dir, 'data.json')
model_dir = os.path.join(root_exp_dir, 'models')
log_dir = os.path.join(root_exp_dir, 'logs')
eval_dir = os.path.join(root_exp_dir, 'evals')

best_checkpoint = os.path.join(model_dir, 'best_model.pth')
final_checkpoint = os.path.join(model_dir, 'final_model.pth')

os.makedirs(f'{workspace_dir}/exps/{model_name}/{data_name}', exist_ok=True)

%cd {workspace_dir}/exps/{model_name}/{data_name}

/nfs/Workspace/TeethSeg/exps/cswinunetr_96_3_mask/tooth


## Train/Test

In [50]:
# training
!PYTHONPATH={workspace_dir} /opt/conda/envs/TeethSeg/bin/python {workspace_dir}/expers/train.py \
--exp_name={exp_name} \
--data_name={data_name} \
--root_exp_dir={root_exp_dir} \
--model_name={model_name}\
--data_dir={data_dir} \
--model_dir={model_dir} \
--log_dir={log_dir} \
--eval_dir={eval_dir} \
--data_dicts_json={data_json} \
--split_train_ratio=0.6 \
--num_fold=3 \
--fold=2 \
--val_every=20 \
--max_early_stop_count=20 \
--max_epoch=20000  \
--out_channels=2 \
--scale_intensity_type='range' \
--a_min=218 \
--a_max=3187 \
--roi_x=96 \
--roi_y=96 \
--roi_z=96 \
--space_x=0.4 \
--space_y=0.4 \
--space_z=0.4 \
--optim='AdamW' \
--lr=1e-4 \
--weight_decay=5e-4 \
--checkpoint={best_checkpoint} \
--no-use_pretrained \
--pin_memory \
# --test_mode

cuda is available
model: cswinunetr_96_3_mask
img_size:  (96, 96, 96)
patch_size:  (2, 2, 2)
window_size:  (7, 7, 7)
optimzer: AdamW
{'lr': 0.0001, 'weight_decay': 0.0005}
<class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.
load json from /nfs/Workspace/dataset/teeth/data50/data.json
train files (30): ['1000966359_20180113', '1000889125_20191101', '1001162439_20180110', '1000983254_20180904', '1001152328_20180112', '1001218388_20180109', '1000915187_20180115', '1001223657_20180112', '1001142392_20180511', '1001142392_20180110', '1001142392_20180116', '1000971031_20180112', '1001022839_20180110', '1001173165_20171205', '1000995722_20180112', '1000889125_20200421', '1001127112_20180109', '1001247962_20180115', '1001133637_20180110', '1001172283_20190622', '1001152328_20180910', '1001232054_20180109', '1001020384_20180114', '1000813648_20180116', '100

## Network Summary

In [ ]:
from torchsummary import summary
import torch

import sys
sys.path.append('/nfs/Workspace/TeethSeg')

In [ ]:
from networks.CSWinTransformer.models.cswinunetr_96_3 import CSwinUNETR as cswinunetr_96_3

In [ ]:
input_shape = (1, 96, 96, 96)

cswt = cswinunetr_96_3(
    img_size=(96, 96, 96), 
    split_size=(1,2,3,3),
    cswt_depths=[2,4,32,2],
    cwst_first_stage_divide=2,
    in_channels=1,
    out_channels=2,
    feature_size=48,
    use_checkpoint=False,
    drop_rate=0.,
    attn_drop_rate=0.,
    dropout_path_rate=0.1
)
s = summary(cswt, input_shape)